In [3]:
!pip install -U ultralytics

In [4]:
!pip install opencv-python

In [6]:
import cv2
import time
import os
from ultralytics import YOLO
import numpy as np

MODEL_WEIGHTS = "yolov8n.pt" 
import torch
DEVICE = "0" if torch.cuda.is_available() else "cpu"

SCORE_THRESHOLD = 0.35       
WIN_NAME = "YOLOv8 Webcam - Detection + Tracking"

OUT_DIR = "yolo_screenshots"
os.makedirs(OUT_DIR, exist_ok=True)

VIDEO_OUT = "12340220_AmayDixit_DSP301_Lab5_LiveWebcam.avi"  
FPS_RECORD = 20.0                     
FOURCC = cv2.VideoWriter_fourcc(*'XVID') 

print("[INFO] Loading model...")
model = YOLO(MODEL_WEIGHTS)

try:
    class_names = model.names
except Exception:
    class_names = {}

print("[INFO] Starting webcam stream. Press 'q' to quit, 's' to save screenshot.")
cap = cv2.VideoCapture(0) 

if not cap.isOpened():
    raise RuntimeError("Failed to open webcam. Check camera permissions or device index.")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

video_writer = cv2.VideoWriter(VIDEO_OUT, FOURCC, FPS_RECORD, (frame_width, frame_height))

cv2.namedWindow(WIN_NAME, cv2.WINDOW_NORMAL)
stream = model.track(source=0, stream=True, device=DEVICE)

frame_ts = time.time()

try:
    for result in stream:
        frame = getattr(result, "orig_img", None)

        if frame is None and hasattr(result, "orig_imgs"):
            if isinstance(result.orig_imgs, (list, tuple)) and len(result.orig_imgs) > 0:
                frame = result.orig_imgs[0]
        if frame is None:
            try:
                frame = result.plot()
            except Exception:
                continue

        out_frame = frame.copy()

        boxes = getattr(result, "boxes", None)
        if boxes is not None:
            try:
                xyxy = boxes.xyxy.cpu().numpy()
            except Exception:
                xyxy = np.array([])
            try:
                confs = boxes.conf.cpu().numpy()
            except Exception:
                confs = np.array([])
            try:
                classes = boxes.cls.cpu().numpy().astype(int)
            except Exception:
                classes = np.array([])
            ids = None
            if hasattr(boxes, "id"):
                try:
                    ids = boxes.id.cpu().numpy().astype(int)
                except Exception:
                    ids = None

            for i, box in enumerate(xyxy):
                x1, y1, x2, y2 = map(int, box[:4])
                conf = float(confs[i]) if i < len(confs) else 0.0
                if conf < SCORE_THRESHOLD:
                    continue
                cls_id = int(classes[i]) if i < len(classes) else -1
                cls_name = class_names.get(cls_id, str(cls_id))
                track_id = ids[i] if (ids is not None and i < len(ids)) else None

                cv2.rectangle(out_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                conf_pct = int(conf * 100)
                id_text = f"ID:{track_id}" if track_id is not None else "ID:-"
                label = f"{cls_name} {conf_pct}% {id_text}"

                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(out_frame, (x1, y1 - 20), (x1 + w, y1), (0, 255, 0), -1)
                cv2.putText(out_frame, label, (x1, y1 - 4),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    
        now = time.time()
        fps = 1.0 / (now - frame_ts) if (now - frame_ts) > 0 else 0.0
        frame_ts = now
        cv2.putText(out_frame, f"FPS: {fps:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

        cv2.imshow(WIN_NAME, out_frame)

    
        video_writer.write(out_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        if key == ord("s"):
            ts = int(time.time())
            fname = os.path.join(OUT_DIR, f"screenshot_{ts}.png")
            cv2.imwrite(fname, out_frame)
            print(f"[INFO] Screenshot saved: {fname}")

except KeyboardInterrupt:
    print("[INFO] Interrupted by user.")
except Exception as e:
    print("[ERROR] Exception occurred:", e)
finally:
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()
    print(f"[INFO] Video saved: {VIDEO_OUT}")
    print("[INFO] Exiting.")


[INFO] Loading model...
[INFO] Starting webcam stream. Press 'q' to quit, 's' to save screenshot.

1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)

0: 480x640 3 persons, 104.4ms
0: 480x640 3 persons, 106.0ms
0: 480x640 3 persons, 98.4ms
0: 480x640 3 persons, 105.4ms
0: 480x640 3 persons, 108.0ms
0: 480x640 3 persons, 113.7ms
0: 480x640 3 persons, 115.8ms
0: 480x640 3 persons, 102.3ms
0: 480x640 3 persons, 116.2ms
0: 480x640 3 persons, 103.8ms
0: 480x640 3 persons, 104.0ms
0: 480x640 3 persons, 102.5ms
0: 480x640 2 persons, 100.0ms
0: 480x640 2 persons, 106.3ms
0: 480x640 1 person, 1 cup, 115.4ms
0: 480x640 2 persons, 1 cup, 105.8ms
0: 480x640 2 persons, 1 cup, 105.8ms
0: 480x640 2 persons, 106.9ms
0: 480x640 2 persons, 1 cup, 95.8ms
0: 480x640 1 person, 100.1ms
0: 480x640 1 person, 98.7ms
0: 480x640 1 person, 109.8ms
0: 480x640 1 person, 105.4ms
0: 480x640 1 person, 1 bottle, 98.6ms
0: 480x640 1 person, 1 bottle, 111.1ms
0: 480x640 3 persons, 116.8ms
0: 480x640 3 persons,